In [ ]:
# !pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import glob
import pprint

def getPublicNmaes(paper_directory):
  dir = paper_directory
  if(paper_directory[-1] != "/"):
    dir += "/"
  # 2020のデータまでパス
  filepath = glob.glob(dir + "*")
  # pprint.pprint(filepath)

  dirname = []
  for path in filepath:
    # そのデータの直上のファイル名を取得
    subdirname = os.path.splitext(os.path.basename(path))[0]
    # print(subdirname)
    dirname.append(subdirname)
    # print(os.path.dirname(filepath[0]))

  # 重複をなくす
  dirname = list(set(dirname))
  pprint.pprint(dirname)
  return dirname

In [ ]:
from gensim.models.doc2vec import Doc2Vec
# from gensim.models.doc2vec import TaggedDocument
import csv
import glob
from matplotlib.pyplot import twinx
import numpy as np
import pprint

root_dir = "/content/drive/MyDrive/zemi_3/doc2vec/"

model = Doc2Vec.load("/content/drive/MyDrive/zemi_3/doc2vec/doc2vec_doc2vec_model.model")
print(model.dv)
# papers_name = model.dv.index_to_key.copy()
# pprint.pprint(papers_name[0:10])

edge_filepath = root_dir + 'node/doc2vec_edgedata_all.csv'
node_filepath = root_dir + 'edge/doc2vec_nodedata_all.csv'
# フォルダ作成
if not os.path.isdir(root_dir + "node"):
  os.makedirs(root_dir + "node")
if not os.path.isdir(root_dir + "edge"):
  os.makedirs(root_dir + "edge")
# 各雑誌の取得個数
each_size = 20
# 閾値
cos_sim_th = 0.6

# 雑誌のディレクトリのパス
public_titles = getPublicNmaes(root_dir + "title_abst_csv/")

csv_dir = root_dir + "title_abst_csv/"
papers_name = []
for title in public_titles:
    with open("{}{}.csv".format(csv_dir, title)) as f:
        csvread = csv.reader(f)
        csvdata = []
        for i, row in enumerate(csvread):
          if(i == 0):
            continue
          if(i > each_size):
            break
          csvdata.append(row[0])
        # for i in range(1, each_size+1):
        #   csvdata.append(csvread[i])
        papers_name += csvdata


pprint.pprint(papers_name[0:10])
# # print()
papers_name_len = len(papers_name)
print(papers_name_len)

# node and edge

# いったん全コサイン類似度を計算
cos_sim_table = [[-2]*papers_name_len for i in range(papers_name_len)]
for i in range(papers_name_len):
    for j in range(papers_name_len):
        if(i != j):
            cos_sim = model.dv.similarity(papers_name[i], papers_name[j])
            cos_sim_table[i][j] = cos_sim


print("make edge data")
# edgeデータの作成
with open(edge_filepath, 'w', newline="") as f:
    writer = csv.writer(f)
    writer.writerow(['Source', 'Target', 'Cosine_similarity'])

    for i in range(papers_name_len):
        k = 5

        while(k > 0):
            max_id = np.argmax(cos_sim_table[i])
            if(cos_sim_table[i][max_id] < -1
               or cos_sim_table[i][max_id] < cos_sim_th):
                break

            k -= 1
            writer.writerow([i, max_id, cos_sim_table[i][max_id]])
            cos_sim_table[i][max_id] = -2


# nodeデータの作成
print("make node data")
with open(edge_filepath, mode="r") as edge_file, open(node_filepath, 'w', newline="") as node_file:
    reader = csv.reader(edge_file)
    from_node = [row[0] for row in reader]
    from_node = from_node[1:]
    to_node = [row[1] for row in reader]
    to_node = to_node[1:]

    nodes = from_node + to_node
    nodes = sorted(list(set(nodes)))

    writer = csv.writer(node_file)
    writer.writerow(['Id', 'Label', "Publication"])

    print("\twrite")
    for index in nodes:
        writer.writerow([index, papers_name[int(index)],
                        public_titles[int(index)//each_size]])

AttributeError: ignored